## Adding convolutional layers

In this step we will switch from plain neural networks to convolutional neural networks and see if this makes any difference.

In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Flatten, Dense

import numpy as np
import math

# fixed random seed to have consistent results
np.random.seed(123)


# dimensions of our images.
img_width, img_height = 300, 300

train_data_dir = 'data/train'
validation_data_dir = 'data/test'
nb_train_samples = 3000
nb_validation_samples = 300
epochs = 5
batch_size = 150

/anaconda/envs/py35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
input_shape = (img_height, img_width, 3)

In [5]:
model = Sequential()

# replacing Dense layer with convolution layer
model.add(Conv2D(32, (3, 3), activation = 'relu', input_shape=input_shape)) 
# Think of max pooling as resizing the image to half its dimesion
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(16, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))

In [6]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 298, 298, 32)      896       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 149, 149, 32)      0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 147, 147, 64)      18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 73, 73, 64)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 341056)            0         
_________________________________________________________________
dense_3 (Dense)              (None, 16)                5456912   
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 17        
Total para

**
Notice that number of trainable parameters drastically decreased from 8 millions to 3.4 millions.
**

In [7]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [8]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 3000 images belonging to 2 classes.
Found 300 images belonging to 2 classes.


In [9]:
model.fit_generator(
    train_generator,
    steps_per_epoch= math.ceil(nb_train_samples / batch_size),
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps= math.ceil(nb_validation_samples/batch_size))

Epoch 1/5
20/20 [==============================] - 63s 3s/step - loss: 0.9279 - acc: 0.5327 - val_loss: 0.6592 - val_acc: 0.5967
Epoch 2/5
20/20 [==============================] - 61s 3s/step - loss: 0.6122 - acc: 0.6657 - val_loss: 0.5945 - val_acc: 0.6833
Epoch 3/5
20/20 [==============================] - 61s 3s/step - loss: 0.5623 - acc: 0.7080 - val_loss: 0.5795 - val_acc: 0.7133
Epoch 4/5
20/20 [==============================] - 61s 3s/step - loss: 0.5259 - acc: 0.7490 - val_loss: 0.5590 - val_acc: 0.7333
Epoch 5/5
20/20 [==============================] - 61s 3s/step - loss: 0.4929 - acc: 0.7617 - val_loss: 0.5313 - val_acc: 0.7400


**
Not bad for just two convolution layers, more than 70% now.
Let's do something different in notebook #5 : Tranfer Learning.
**